# 1) Load the Monet_to_Photo Dataset using Pytorch. monet_to_photo (https://drive.google.com/drive/folders/1KkSYKqdmO1GE6gSP38UCsOrYZ7QUGj-x?usp=drive_link)

# 2) Create and train Cycle GAN on this dataset.
# 3) Show your results( Few converted images).
# 4) load your favorite image (or your picture) apply Cyclegan and upload it with the report.

In [2]:
!pip install torch_snippets torch_summary --quiet

In [3]:
import itertools
from PIL import Image
from torch_snippets import *
from torchvision import transforms
from torchvision.utils import make_grid
from torchsummary import summary

In [ ]:
img = Image.open('testA/n02381460_1000.jpg')
img.size